In [1]:
pwd

u'/home/zw/Documents/project/ad-contest/xgboost_lzw'

In [2]:
cd ..

/home/zw/Documents/project/ad-contest


In [3]:
import pandas as pd
import numpy as np

In [4]:
train1 = pd.read_csv('note_book_code/middle_file/train1.csv')
test1 = pd.read_csv('note_book_code/middle_file/test1.csv')

In [10]:
train1.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'label', u'clickTime',
       u'conversionTime', u'creativeID', u'userID', u'positionID',
       u'connectionType', u'telecomsOperator', u'adID', u'camgaignID',
       u'advertiserID', u'appID', u'appPlatform', u'user_count', u'popularity',
       u'age', u'gender', u'education', u'marriageStatus', u'haveBaby',
       u'hometown', u'residence', u'appcount', u'love_app'],
      dtype='object')

In [11]:
from utils import config
data_root = config.get_home_dir() + 'raw_data/pre'
dfAd = pd.read_csv("%s/ad.csv"%data_root)
df_app_cat = pd.read_csv('%s/app_categories.csv'%data_root)

In [14]:
position = pd.read_csv("%s/position.csv"%data_root)

In [ ]:
predectors = [u'creativeID', u'userID', u'positionID',
       u'connectionType', u'telecomsOperator', u'adID', u'camgaignID',
       u'advertiserID', u'appID', u'appPlatform', u'user_count', u'popularity',
       u'age', u'gender', u'education', u'marriageStatus', u'haveBaby',
       u'hometown', u'residence', u'appcount', u'love_app']

In [17]:
def merge_t(train,test):
    train = pd.merge(train,df_app_cat,how='left',on='appID')
    test = pd.merge(test,df_app_cat,how='left',on='appID')
    train = pd.merge(train,position,how='left',on='positionID')
    test = pd.merge(test,position,how='left',on='positionID')
    return train,test

In [18]:
train1,test1 = merge_t(train1,test1)

In [20]:
train1.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'label', u'clickTime',
       u'conversionTime', u'creativeID', u'userID', u'positionID',
       u'connectionType', u'telecomsOperator', u'adID', u'camgaignID',
       u'advertiserID', u'appID', u'appPlatform', u'user_count', u'popularity',
       u'age', u'gender', u'education', u'marriageStatus', u'haveBaby',
       u'hometown', u'residence', u'appcount', u'love_app', u'appCategory',
       u'sitesetID', u'positionType'],
      dtype='object')

In [29]:
predictors4 = [u'connectionType', u'telecomsOperator', u'adID', u'camgaignID',
       u'advertiserID', u'appID', u'appPlatform', u'user_count', u'popularity',
       u'age', u'gender', u'education', u'marriageStatus', u'haveBaby',
       u'hometown', u'residence', u'appcount', u'love_app', u'appCategory',
       u'sitesetID', u'positionType']
predictors1 = [u'connectionType', u'telecomsOperator', u'adID', u'camgaignID',
       u'advertiserID', u'appID', u'appPlatform',
       u'age', u'gender', u'education', u'marriageStatus', u'haveBaby',
       u'hometown', u'residence',  u'appCategory',
       u'sitesetID', u'positionType']
predictors2 = [u'connectionType', u'telecomsOperator', u'adID', u'camgaignID',
       u'advertiserID', u'appID', u'appPlatform', u'user_count', u'popularity',
       u'age', u'gender', u'education', u'marriageStatus', u'haveBaby',
       u'hometown', u'residence', u'appCategory',
       u'sitesetID', u'positionType']
predictors3 = [u'connectionType', u'telecomsOperator', u'adID', u'camgaignID',
       u'advertiserID', u'appID', u'appPlatform',
       u'age', u'gender', u'education', u'marriageStatus', u'haveBaby',
       u'hometown', u'residence', u'appcount', u'love_app', u'appCategory',
       u'sitesetID', u'positionType']

In [22]:
train1.to_csv(config.get_file_path('v1','xgboost','v4_lzw') + 'train1.csv')
test1.to_csv(config.get_file_path('v1','xgboost','v4_lzw') + 'test1.csv')

In [25]:
import xgboost as xgb

In [30]:
dtrain1 = xgb.DMatrix(train1[predictors1].values,label=train1['label'])

In [33]:
bst = xgb.train(plst, dtrain1, num_round)
bst.save_model(config.get_file_path('v1','xgboost','v4_lzw')+'first_model')

In [34]:
dtest1 = xgb.DMatrix(test1[predictors1].values)

In [38]:
from xgboost.sklearn import XGBClassifier

In [39]:
alg = XGBClassifier(bst)

In [50]:
alg.get_params()['n_estimators']

100

In [66]:
train2 = pd.read_csv('note_book_code/middle_file/train2.csv') #2
test2 = pd.read_csv('note_book_code/middle_file/test2.csv')  #2
train2,test2 = merge_t(train2,test2) #4

train2.to_csv(config.get_file_path('v1','xgboost','v4_lzw') + 'train2.csv') #2
test2.to_csv(config.get_file_path('v1','xgboost','v4_lzw') + 'test2.csv') #2
dtrain2 = xgb.DMatrix(train2[predictors2].values,label=train2['label']) # 4
bst = xgb.train(plst, dtrain2, num_round) #1
bst.save_model(config.get_file_path('v1','xgboost','v4_lzw')+'2_model') #1
dtest2 = xgb.DMatrix(test2[predictors2].values) # 3

In [67]:
y_test2 = bst.predict(dtest2)

In [68]:
y_test2

array([ 0.02698107,  0.04067388,  0.03578694, ...,  0.06837387,
        0.03911898,  0.02612962], dtype=float32)

In [69]:
df2 = pd.DataFrame({"instanceID": test2["instanceID"].values, "proba": y_test2})

In [74]:
train4 = pd.read_csv('note_book_code/middle_file/train4.csv') #2
test4 = pd.read_csv('note_book_code/middle_file/test4.csv')  #2
train4,test4 = merge_t(train4,test4) #4

train4.to_csv(config.get_file_path('v1','xgboost','v4_lzw') + 'train4.csv') #2
test4.to_csv(config.get_file_path('v1','xgboost','v4_lzw') + 'test4.csv') #2
dtrain4 = xgb.DMatrix(train4[predictors4].values,label=train4['label']) # 4
bst = xgb.train(plst, dtrain4, num_round) #1
bst.save_model(config.get_file_path('v1','xgboost','v4_lzw')+'4_model') #1
dtest4 = xgb.DMatrix(test4[predictors4].values) # 3
y_test4 = bst.predict(dtest4)  #2
df4 = pd.DataFrame({"instanceID": test4["instanceID"].values, "proba": y_test4}) #3
df4

,instanceID,proba
0,3,0.027756
1,8,0.089208
2,10,0.040314
3,12,0.050021
4,13,0.024893
5,14,0.046647
6,15,0.048219
7,19,0.065736
8,20,0.043056
9,21,0.065736


In [76]:
df1.shape

(33258, 2)

In [77]:
df2.shape

(130729, 2)

In [78]:
df11 = df1.copy()
df22 = df2.copy()

In [86]:
c = pd.concat([df11,df22,df3,df4])

In [87]:
c.shape[0] == df1.shape[0] + df2.shape[0] + df3.shape[0]+df4.shape[0]

True

In [89]:
c.shape

(338489, 2)

In [91]:
c.sort_values('instanceID',inplace=True)

In [93]:
import zipfile
c.to_csv("submission.csv", index=False)
with zipfile.ZipFile(config.get_file_path('v1','xgboost','v4_lzw')+"submission.zip", "w") as fout:
        fout.write("submission.csv", compress_type=zipfile.ZIP_DEFLATED)

In [ ]:
def modelfit(alg, dtrain, dtest, predictors, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        xgtest = xgb.DMatrix(dtest[predictors].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])

    # 建模
    alg.fit(dtrain[predictors], dtrain['Disbursed'], eval_metric='auc')

    # 对训练集预测
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:, 1]

    # 输出模型的一些结果
    print "\n关于现在这个模型"
    print "准确率 : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions)
    print "AUC 得分 (训练集): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob)

    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [97]:
testfull = pd.read_csv('note_book_code/middle_file/test_full.csv')

In [101]:
ori_path = config.get_origin_path()
user_apps = pd.read_csv(ori_path + 'user_installedapps.csv')
ua_ac = pd.read_csv(ori_path + 'user_app_actions.csv')

In [112]:
cheat = pd.merge(testfull,ua_ac,how='left',on=['userID','appID'])

In [113]:
type(cheat['installTime'])

pandas.core.series.Series

In [114]:
cheat.loc[cheat['installTime'].isnull() == False].shape

(2259, 26)

In [118]:
cheat2 = cheat.loc[cheat['installTime'].isnull() == False]

In [121]:
cheat2  = cheat2[['instanceID','installTime']]

In [124]:
cheat= pd.merge(c,cheat2,how='left',on='instanceID')

In [140]:
intime[0] = -1
intime

0        -1.0
1        -1.0
2        -1.0
3        -1.0
4        -1.0
5        -1.0
6        -1.0
7        -1.0
8        -1.0
9        -1.0
10       -1.0
11       -1.0
12       -1.0
13       -1.0
14       -1.0
15       -1.0
16       -1.0
17       -1.0
18       -1.0
19       -1.0
20       -1.0
21       -1.0
22       -1.0
23       -1.0
24       -1.0
25       -1.0
26       -1.0
27       -1.0
28       -1.0
29       -1.0
         ... 
338459   -1.0
338460   -1.0
338461   -1.0
338462   -1.0
338463   -1.0
338464   -1.0
338465   -1.0
338466   -1.0
338467   -1.0
338468   -1.0
338469   -1.0
338470   -1.0
338471   -1.0
338472   -1.0
338473   -1.0
338474   -1.0
338475   -1.0
338476   -1.0
338477   -1.0
338478   -1.0
338479   -1.0
338480   -1.0
338481   -1.0
338482   -1.0
338483   -1.0
338484   -1.0
338485   -1.0
338486   -1.0
338487   -1.0
338488   -1.0
Name: installTime, Length: 338489, dtype: float64

In [129]:
prob = cheat['proba']
intime = cheat['installTime']

In [133]:
intime = intime.fillna(-1)

In [141]:
for i in range(prob.size):
    if intime[i] > 0:
        print prob[i]
        prob[i] =0
        print prob[i]

0.0273566
0.0
0.0528533
0.0
0.0403503
0.0
0.0487785
0.0
0.0259777
0.0
0.0532432
0.0


/home/zw/Documents/project/virtualenv_home/webclassify/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.16554
0.0
0.159921
0.0
0.0590722
0.0
0.0454361
0.0
0.0481616
0.0
0.045114
0.0
0.053529
0.0
0.167669
0.0
0.0420264
0.0
0.047616
0.0
0.047616
0.0
0.0259517
0.0
0.0259517
0.0
0.0430556
0.0
0.047616
0.0
0.0944398
0.0
0.105408
0.0
0.047664
0.0
0.0427405
0.0
0.0417285
0.0
0.0420264
0.0
0.0517203
0.0
0.044156
0.0
0.0403503
0.0
0.177018
0.0
0.0257736
0.0
0.0500208
0.0
0.0380299
0.0
0.0408238
0.0
0.0408238
0.0
0.0342945
0.0
0.0248928
0.0
0.162556
0.0
0.0487785
0.0
0.0642086
0.0
0.0366877
0.0
0.047616
0.0
0.0272684
0.0
0.0593448
0.0
0.0532432
0.0
0.0500208
0.0
0.047616
0.0
0.0532432
0.0
0.0487785
0.0
0.0313248
0.0
0.246692
0.0
0.0507684
0.0
0.0420264
0.0
0.0425999
0.0
0.0277556
0.0
0.0528533
0.0
0.0272684
0.0
0.163057
0.0
0.0571561
0.0
0.211369
0.0
0.0436455
0.0
0.192661
0.0
0.0487785
0.0
0.0483894
0.0
0.0408238
0.0
0.0507684
0.0
0.0463203
0.0
0.246692
0.0
0.0430556
0.0
0.0487785
0.0
0.18268
0.0
0.18268
0.0
0.18268
0.0
0.0581003
0.0
0.0598959
0.0
0.0385445
0.0
0.0449693
0.0
0.0366877
0.0
0.048

In [135]:
intime.shape

(338489,)

In [145]:
c['proba'] = prob.values

In [148]:
c.to_csv("submission.csv", index=False)
with zipfile.ZipFile(config.get_file_path('v1','xgboost','v4_lzw')+"submission2.zip", "w") as fout:
        fout.write("submission.csv", compress_type=zipfile.ZIP_DEFLATED)

In [150]:
cheat

,instanceID,proba,installTime
0,1,0.024261,NaN
1,2,0.024169,NaN
2,3,0.027756,NaN
3,4,0.026981,NaN
4,5,0.024169,NaN
5,6,0.040674,NaN
6,7,0.035787,NaN
7,8,0.089208,NaN
8,9,0.024261,NaN
9,10,0.040314,NaN


In [152]:
cheat.to_csv('note_book_code/middle_file/cheat.csv')

In [154]:
trainfull = pd.read_csv('note_book_code/middle_file/train_full.csv')

In [157]:
trainfull.fillna(-1)

,Unnamed: 0,Unnamed: 0.1,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator,...,popularity,age,gender,education,marriageStatus,haveBaby,hometown,residence,appcount,love_app
0,0,0,0,170000,-1.0,3089,2798058,293,1,1,...,3.0,25,2,1,3,1,0,1301,44.0,2.0
1,1,1,0,170000,-1.0,1259,463234,6161,1,2,...,-1.0,20,2,1,1,0,0,213,-1.0,-1.0
2,2,2,0,170000,-1.0,4465,1857485,7434,4,1,...,3.0,16,1,1,0,0,0,1502,-1.0,-1.0
3,3,3,0,170000,-1.0,1004,2038823,977,1,1,...,3.0,0,0,0,0,0,0,1001,69.0,3.0
4,4,4,0,170000,-1.0,1887,2015141,3688,1,1,...,6.0,35,1,2,2,0,1001,1001,-1.0,-1.0
5,5,5,0,170000,-1.0,3293,1177829,3347,1,1,...,-1.0,30,2,1,2,1,107,107,-1.0,-1.0
6,6,6,0,170000,-1.0,4793,1257450,1876,1,2,...,-1.0,24,2,2,1,0,0,0,28.0,1.0
7,7,7,0,170000,-1.0,1456,764204,6086,1,3,...,3.0,15,2,3,0,0,308,308,78.0,4.0
8,8,8,0,170000,-1.0,4465,1438585,2426,2,3,...,3.0,19,1,1,1,0,803,601,-1.0,-1.0
9,9,9,0,170000,-1.0,985,1750317,5414,1,3,...,3.0,19,1,1,1,0,2102,2102,88.0,4.0


In [158]:
testfull = testfull.fillna(-1)

,Unnamed: 0,Unnamed: 0.1,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator,...,popularity,age,gender,education,marriageStatus,haveBaby,hometown,residence,appcount,love_app
0,0,0,0,170000,NaN,3089,2798058,293,1,1,...,3.0,25,2,1,3,1,0,1301,44.0,2.0
1,1,1,0,170000,NaN,1259,463234,6161,1,2,...,NaN,20,2,1,1,0,0,213,NaN,NaN
2,2,2,0,170000,NaN,4465,1857485,7434,4,1,...,3.0,16,1,1,0,0,0,1502,NaN,NaN
3,3,3,0,170000,NaN,1004,2038823,977,1,1,...,3.0,0,0,0,0,0,0,1001,69.0,3.0
4,4,4,0,170000,NaN,1887,2015141,3688,1,1,...,6.0,35,1,2,2,0,1001,1001,NaN,NaN
5,5,5,0,170000,NaN,3293,1177829,3347,1,1,...,NaN,30,2,1,2,1,107,107,NaN,NaN
6,6,6,0,170000,NaN,4793,1257450,1876,1,2,...,NaN,24,2,2,1,0,0,0,28.0,1.0
7,7,7,0,170000,NaN,1456,764204,6086,1,3,...,3.0,15,2,3,0,0,308,308,78.0,4.0
8,8,8,0,170000,NaN,4465,1438585,2426,2,3,...,3.0,19,1,1,1,0,803,601,NaN,NaN
9,9,9,0,170000,NaN,985,1750317,5414,1,3,...,3.0,19,1,1,1,0,2102,2102,88.0,4.0
